In [1]:
import os
# import lxml
from lxml import etree
import pandas as pd
import numpy as np

In [41]:
import sys
from collections import defaultdict

In [21]:
from sklearn.metrics import classification_report

In [3]:
import os

import gensim
import pymorphy2
import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn import preprocessing

In [4]:
from sklearn.tree import DecisionTreeClassifier

In [5]:
from sklearn.preprocessing import Normalizer

In [6]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield gensim.models.doc2vec.LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])


In [50]:
cl = ['заметка','интервью','отчет']

In [51]:
f = open('out.csv', 'r')

file_hash = {}
for line in f:
    attrs = line.split(';')
    path = attrs[0]
    class_name = attrs[8]
    if ' ' in class_name:
        class_name = class_name.split()[0]
    if '|' in class_name:
        class_name = class_name.split('|')[0]
    filename = os.path.basename(path)
    if class_name in cl:
        file_hash[filename] = class_name


In [44]:
def check_len_of_classes():
    f = open('out.csv', 'r')
    dict_len = defaultdict(int)
    for line in f:
        attrs = line.split(';')
        path = attrs[0]
        class_name = attrs[8]
        dict_len[class_name] += 1
    return dict_len

In [52]:
#  check_len_of_classes() 

In [53]:
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [54]:
def read_txt(path):
    for root, dirs, files in os.walk(path):
        for filename in files:
            open_name = os.path.join(root, filename)
            if filename in file_hash:
                class_text = file_hash[filename]
                tree = etree.parse(open_name)
                yield tree, class_text

In [55]:
sent = []
for tree, class_text in read_txt('/Users/Valeriya/Desktop/CoursePaper/corpus'):
    all_p = tree.xpath('//p')
    joined = '\n'.join([p.text for p in all_p if p.text is not None])
    sent.append((joined, class_text))

In [56]:
len(sent)

2506

In [57]:
only_texts = [text[0] for text in sent]
classes = [text[1] for text in sent]

In [58]:
set(classes)

{'заметка', 'интервью', 'отчет'}

In [61]:
docLabels = [str(i) for i in range(2506)]

it = DocIterator(only_texts, docLabels)

model = gensim.models.Doc2Vec(window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
for epoch in range(10):
    model.train(it)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it)

In [22]:
# model.infer_vector(['Мама', 'мыла', 'раму'])

In [62]:
len(model.docvecs)

2506

In [27]:
# gensim.models.Doc2Vec(it, size=100, window=8, min_count=5, workers=4)

In [63]:
X_train, X_test, y_train, y_test = train_test_split(list(model.docvecs), classes, test_size=0.3)

In [64]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

    заметка       0.81      0.95      0.87       527
   интервью       0.56      0.34      0.42       109
      отчет       0.50      0.31      0.38       116

avg / total       0.73      0.76      0.73       752



In [65]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print(classification_report(y_test, y_pred_svc))

             precision    recall  f1-score   support

    заметка       0.70      1.00      0.82       527
   интервью       0.00      0.00      0.00       109
      отчет       0.00      0.00      0.00       116

avg / total       0.49      0.70      0.58       752



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:
regr = LogisticRegression()
regr.fit(X_train, y_train)
y_pred_regr = regr.predict(X_test)
print(classification_report(y_test, y_pred_regr))

             precision    recall  f1-score   support

    заметка       0.85      0.95      0.90       527
   интервью       0.80      0.60      0.68       109
      отчет       0.69      0.51      0.59       116

avg / total       0.82      0.83      0.82       752



In [67]:
y_pred_rest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).predict(X_test)
print(classification_report(y_test, y_pred_rest))

             precision    recall  f1-score   support

    заметка       0.86      0.91      0.89       527
   интервью       0.63      0.61      0.62       109
      отчет       0.60      0.47      0.52       116

avg / total       0.79      0.80      0.79       752



Gradient Descent

In [82]:
clf_grad = linear_model.SGDClassifier()
clf_grad.fit(X_train, y_train)
y_pred_grad = clf_grad.predict(X_test)
print(classification_report(y_test, y_pred_grad))

             precision    recall  f1-score   support

    заметка       0.82      0.91      0.86       527
   интервью       0.63      0.50      0.56       109
      отчет       0.60      0.39      0.47       116

avg / total       0.76      0.77      0.76       752



In [83]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_norm = min_max_scaler.fit_transform(X_train)
X_test_norm = min_max_scaler.fit_transform(X_test)

clf_bayes = MultinomialNB()
clf_bayes.fit(X_train_norm, y_train)
y_pred_bayes = clf_bayes.predict(X_test_norm)
print(classification_report(y_test, y_pred_bayes))

             precision    recall  f1-score   support

    заметка       0.70      1.00      0.82       527
   интервью       0.00      0.00      0.00       109
      отчет       0.00      0.00      0.00       116

avg / total       0.49      0.70      0.58       752



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [84]:
clf_tree = DecisionTreeClassifier(random_state=0)
clf_tree.fit(X_train, y_train)
y_pred_tree = clf_tree.predict(X_test)
print(classification_report(y_test, y_pred_tree))

             precision    recall  f1-score   support

    заметка       0.82      0.80      0.80       527
   интервью       0.43      0.37      0.40       109
      отчет       0.22      0.28      0.25       116

avg / total       0.67      0.65      0.66       752

